# 自前のネットワークを組んで学習させてみる

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
plt.rcParams['font.family'] = 'Meiryo'

In [2]:
train_fn = r'./dataset_cur_train.csv'
test_fn = r'./dataset_cur_test.csv'

In [3]:
df_train = pd.read_csv(train_fn,index_col=0)
df_train['red_diff'] = df_train['red_diff'].astype(np.float32)
df_train['remain_ends'] = df_train['remain_ends'].astype(np.float32)
df_train['last_stone_is_red'] = df_train['last_stone_is_red'].astype(np.float32)
df_train['red_postion'] = df_train['red_postion'].astype(np.float32)
display(df_train.dtypes)
display(df_train.head(3))

red_diff             float32
remain_ends          float32
last_stone_is_red    float32
red_postion          float32
filepath              object
dtype: object

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
8,1.0,0.0,1.0,3.0,./dataset_o\ECC2023_ResultsBook_Women_A-Divisi...
0,2.0,8.0,1.0,0.0,./dataset_o\ECC2021_ResultsBook_Men_A-Division...
8,-2.0,0.0,0.0,3.0,./dataset_o\ECC2019_ResultsBook_Women_A-Divisi...


## red_diff が目的、fn,remain_ends,last_stone_is_red,red_postion が説明変数
説明変数を標準化する

In [4]:
# 標準化
stdsc = StandardScaler()
##学習時の標準化したパラメータは、評価、本番時におなじ重みで標準化する処理が必要
x_train_df = df_train.copy().drop(['filepath','red_diff'],axis=1)
x_train_std = stdsc.fit_transform(x_train_df)
display( x_train_std[:3] )
## DataFrameの値を入れ替え
qcl = df_train.columns.to_list()
qcl.remove('filepath')
qcl.remove('red_diff')
print(qcl)
df_train[qcl] = x_train_std
pickle.dump(stdsc, open("stdsc_02240209.pkl", "wb"))
display(stdsc.n_features_in_, stdsc.mean_ , stdsc.var_) 
df_train.head(5)

array([[-1.5558919 ,  0.9972611 ,  1.2323233 ],
       [ 1.5270197 ,  0.9972611 , -0.01566701],
       [-1.5558919 , -1.0027465 ,  1.2323233 ]], dtype=float32)

['remain_ends', 'last_stone_is_red', 'red_postion']


3

array([4.0374607 , 0.50137133, 0.03766138])

array([6.73376293, 0.24999812, 5.7785669 ])

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
8,1.0,-1.555892,0.997261,1.232323,./dataset_o\ECC2023_ResultsBook_Women_A-Divisi...
0,2.0,1.527020,0.997261,-0.015667,./dataset_o\ECC2021_ResultsBook_Men_A-Division...
8,-2.0,-1.555892,-1.002746,1.232323,./dataset_o\ECC2019_ResultsBook_Women_A-Divisi...
7,1.0,-1.170528,0.997261,-0.431664,./dataset_o\OWG2018_ResultsBook\geme835end8.png
8,-3.0,-1.555892,-1.002746,0.400330,./dataset_o\WWCC2018_ResultsBook\geme338end9.png


### 20240209_samplescreeningで画像の平均 分散は計算ずみ
平均:[0.5254706740379333, 0.5207158327102661, 0.5373605489730835] 分散[0.47727182507514954, 0.4752059578895569, 0.4878864884376526] :]

In [5]:
import torch
from torchvision import transforms
import torchvision


In [6]:
## 画像変換の定義
w,h = Image.open(df_train['filepath'].values[0]).size
## 正方形にするための差分
pad = (h-w)/2
display([w,h],pad)
target_size = 224
## train用
transform_train = transforms.Compose([
    transforms.Pad(( 240 // 2, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.RandomHorizontalFlip(0.33),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5254, 0.521, 0.538], std=[0.477, 0.475, 0.487])
])
# valid/test用
transform_test = transforms.Compose([
    transforms.Pad(( 240 // 2, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.RandomHorizontalFlip(0.33),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5254, 0.521, 0.538], std=[0.477, 0.475, 0.487])
])

[300, 540]

120.0

In [7]:
from torch.utils.data import DataLoader, Dataset

# 自前のデータセット定義

In [8]:
class ImgValueDataset(Dataset):
    def __init__(self, df, classcol , fncol , transform):
        
        ##self.label_list  = df[classcol].to_list()
        self.label_list  = pd.get_dummies(df[classcol]).values
        self.img_pathlist  = df[fncol].to_list()
        cols = df.columns.to_list()
        cols.remove(fncol)
        cols.remove(classcol)
        self.x_values = df[cols].values
        self.transform = transform

    def __len__(self):  
        return len( self.img_pathlist )
    
    def __getitem__(self, index):
        # 画像をPILとして読み込む
        #print(index)
        image = Image.open(self.img_pathlist[index])
        
        label = self.label_list[index]
                             
        if self.transform is not None:
            ##print('use transform')
            image = self.transform(image)
        return image, label 
    pass

In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_train, test_size=0.2, stratify=df_train['red_diff'])
train_dataset = ImgValueDataset( train_df ,classcol='red_diff' , fncol='filepath',transform=transform_train)
test_dataset = ImgValueDataset( test_df ,classcol='red_diff' , fncol='filepath' ,transform=transform_test)

In [10]:
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


In [11]:
images, labels = next(iter(train_loader))
images.shape , labels

(torch.Size([4, 3, 224, 224]),
 tensor([[False, False, False, False, False, False, False,  True, False, False,
          False],
         [False, False, False, False, False,  True, False, False, False, False,
          False],
         [False, False, False, False, False, False,  True, False, False, False,
          False],
         [False, False, False, False, False, False,  True, False, False, False,
          False]]))

In [12]:
class_list = train_df['red_diff'].unique()
class_list , len(class_list)

(array([-2.,  0.,  1.,  2., -1., -3., -4.,  3., -5.,  5.,  4.],
       dtype=float32),
 11)

試しに汎用モデルを読む

In [13]:
from torchvision import models 
import pprint
from torchvision.models import MobileNetV2
Model = models.mobilenet_v2(pretrained=True)
Model


C:\Users\ghost\miniconda3\envs\dl_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ghost\miniconda3\envs\dl_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

# ネットを組む  
https://qiita.com/poorko/items/c151ff4a827f114fe954

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
# CNNモデルの定義
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        
        self.fc1 = nn.Linear(24*106*106, 10240)
        self.Dropout1 = nn.Dropout(p=0.2, inplace=False)
        self.fc2 = nn.Linear(10240, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, input):
      
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))   
        ###print(output.shape)
        output = output.view(-1, 24*106*106)
        output = F.relu(self.fc1(output))  # fc1->relu
        output = self.Dropout(output)
        output = F.relu(self.fc2(output))  # fc2->relu
        output = self.fc3(output)
        
        return output

In [16]:
net = SimpleCNN(11)
print(net)

SimpleCNN(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=269664, out_features=10240, bias=True)
  (Dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=10240, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=11, bias=True)
)


## 損失関数とオプティマイザ

In [17]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

## 訓練（ネットワークが通るか確認）

In [18]:
from torch.autograd import Variable
def saveModel():
    path = "./20240209.Model.pth"
    torch.save(model.state_dict(), path)
    
def testAccuracy(model,test_loader):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

## 学習可能か確認

In [23]:
len(train_dataset)/64

186.859375

In [20]:
%%time
num_epochs = 2
best_accuracy = 0.0
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,drop_last = True)
test_loader = DataLoader(test_df, batch_size=batch_size, shuffle=True, num_workers=0)

# Define your execution device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")
net.to(device)
min_loss = 999999999
for epoch in range(num_epochs):  # loop over the dataset multiple times
    train_loss = 0.0;val_loss = 0.0
    train_batches = 0;val_batches = 0
    running_loss = 0.0 
    net.train()  # 訓練モード
    for index_minbatch , (inputs, labels) in enumerate(train_loader, 0):
        #print(f'min_batch: start:{index_minbatch}')
        inputs ,labels_float  = inputs.to(device) , labels.float().to(device)
        # 勾配のリセット
        optimizer.zero_grad()
        
        outputs = net(inputs)    # 順方向計算
        loss = criterion(outputs, labels_float)   # 損失の計算
        loss.backward()                     # 逆方向計算(勾配計算)
        optimizer.step()                    # パラメータの更新  

        # 損失関数の変化を１0ミニバッチごとに表示
        running_loss += loss.item()     # extract the loss value
        if index_minbatch % 10 == 9:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, index_minbatch + 1, running_loss / 10))
            # zero the loss
            running_loss = 0.0
            pass
    #########################################
    accuracy = testAccuracy(net,test_loader)
    print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
    # we want to save the model if the accuracy is the best
    if accuracy > best_accuracy:
        saveModel()
        best_accuracy = accuracy

print('Finished Training')

The model will be running on cuda:0 device


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.67 GiB is allocated by PyTorch, and 46.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [21]:
outputs.dtype , labels_float.dtype ,labels.dtype

NameError: name 'outputs' is not defined

In [ ]:
outputs.shape , labels_float.shape

In [ ]:
len(train_loader)

In [ ]:
labels_float